Crawling Documents


Set up Elastic Search with your teammates to have the same cluster name and the same index name.

Your crawler will manage a frontier of URLs to be crawled. The frontier will initially contain just your seed URLs. URLs will be added to the frontier as you crawl, by finding the links on the web pages you crawl.

You should crawl at least 20,000 documents individually, including your seed URLs. This will take several hours, so think carefully about how to adequately test your program without running it to completion in each debugging cycle.
You should choose the next URL to crawl from your frontier using a best-first strategy. See Frontier Management, below.
Your crawler must strictly conform to the politeness policy detailed in the section below. You will be consuming resources owned by the web sites you crawl, and many of them are actively looking for misbehaving crawlers to permanently block. Please be considerate of the resources you consume.
You should only crawl HTML documents. It is up to you to devise a way to ensure this. However, do not reject documents simply because their URLs don’t end in .html or .htm.
You should find all outgoing links on the pages you crawl, canonicalize them, and add them to your frontier if they are new. See the Document Processing and URL Canonicalization sections below for a discussion.
For each page you crawl, you should store the following filed with ElasticSearch : an id, the URL, the HTTP headers, the page contents cleaned (with term positions), the raw html, and a list of all in-links (known) and out-links for the page.
Once your crawl is done, you should get together with your teammates and figure out how to merge the indexes. With proper ids, the ElasticSearch will do the merging itself, you still have to manage the link graph.

In [1]:
import re
import os
import time
import urllib
import pickle
import requests
from collections import deque
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import urllib.robotparser 
import w3lib.url
    from bs4.element import Comment
from collections import OrderedDict
from lxml import etree as ET
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [28]:
class invalid_extension():
    def __init__(self):
        self.dict_of_invalid_extension = {"pdf" : True,
                                          "jpeg" : True,
                                          "jpg" : True,
                                          "tiff" : True,
                                          "gif" : True,
                                          "bmp" : True,
                                          "png" : True,
                                          "svg": True}
        self.list_of_words = ["costa",
                              "concordia",
                              "titanic",
                              "shipwreck",
                              "ship",
                              "disaster",
                              "tragedy",
                              "sink", 
                              "maritime",
                              "accident"]
        self.social_media = [
                             "facebook",
                             "reddit",
                             "linkedin",
                             "twitter",
                             "instagram",
                             "pinterest",
                             "accounts.google",
                             "plus.google",
                             "youtube",
                             "amazon",
                             "tumblr",
                             "membership",
                             "messenger",
                             "whatsapp",
                              ]
        self.wikipedia_invalid = [
                                "/wiki/Portal",
                                "/wiki/Template",
                                "/wiki/Talk",
                                "/wiki/List",
                                "/wiki/Category",
                                "/wiki/Help",
                                "/wiki/File",
                                "/wiki/Wikipedia",
                                "/wiki/Special",
                                "/wiki/Main_Page",
                                "/w/", "wikimedia",
                                "mediawiki",
                                "aboutus"
                                ]
        

        self.oot_telegraph_path_list = [
                                       "/sponsored/",
                                       "/sport/",
                                       "/lifestyle/",
                                       "/advertising/",
                                       "/topics/",
                                       "/subscriptions/",
                                       "/promotions/",
                                       "/syndication/",
                                       "/expat",
                                       "/film",
                                       "/technology",
                                       "/fashion",
                                       "/motoring",
                                       "/cars",
                                       "/luxury",
                                       "/travel",
                                       "/women",
                                       "/termsandconditions",
                                       "/telegraphtv",
                                       "/football",
                                       "/culture",
                                       "/environment",
                                       "/business",
                                       "/stage",
                                       "/tv-and-radio",
                                       "/education",
                                       "/journalists",
                                       "/profile",
                                       "/financialservices/"
                                        ]

        self.oot_telegraph_subdomains = [
                                       "tickets.telegraph.co.uk",
                                       "puzzles.telegraph.co.uk",
                                       "announcements.telegraph.co.uk",
                                       "jobs.telegraph.co.uk",
                                       "blogs.telegraph.co.uk",
                                       "subscriber.telegraph.co.uk",
                                       "fantasycricket.telegraph.co.uk",
                                       "begambleaware.org",
                                       "gamcare.org.uk",
                                       "secure.gamblingcommission.gov.uk",
                                       "fantasyfootball.telegraph.co.uk",
                                       "coffeevsgangs.telegraph.co.uk",
                                       "bit.ly",
                                       "coffeevsgangs.com",
                                       "coffeemadehappy.com",
                                       "fantasycricket.telegraph.co.uk",
                                       "fantasyfootballscout.co.uk",
                                       "workforus.theguardian.com",
                                        "theasylum.cc/blog",
                                        "shipwrecklog.com",
                                        "https://www.ship-technology.com/",
                                        "www.patrollerairships.com",
                                        "turkishnavyshipbucket.blogspot.com",
                                        "http://nzshipmarine.com",
                                        "http://2012.tiff-jp.net",
                                        "helsinki", 
                                        "screendaily",
                                        "maritimehistoryofthegreatlakes"
                                       ]

        self.combined = self.social_media + self.wikipedia_invalid + self.oot_telegraph_path_list + self.oot_telegraph_subdomains
        self.expression_wikipedia_social_invalid = re.compile(("|".join(self.combined)),re.I)
        self.keyword_expression = re.compile("|".join(self.list_of_words),re.I)
        
    def valid(self, url):
        if (url.split(".")[-1]).lower() not in self.dict_of_invalid_extension:
            if not re.search(self.expression_wikipedia_social_invalid, url):
                return True
        return False
    
    def keyword_presence(self, text):
        if re.search(self.keyword_expression, text):
            return True
        return False


In [42]:
class crawler_class():
    def __init__(self, counter = 1, url_out_link_dict = {}, url_in_link_dict = {}, depth_dict = {}, queue_dict = OrderedDict(), queue_dict_crawled = {}, last_seen_url = None, max_inlink = None, dict_domain_check = {}):
        self.counter = counter
        self.url_out_link_dict = url_out_link_dict
        self.url_in_link_dict = url_in_link_dict
        self.queue_dict =  queue_dict
        self.queue_dict_crawled = queue_dict_crawled
        self.last_seen_url = last_seen_url 
        self.max_inlink = max_inlink  
        self.dict_domain_check = dict_domain_check
        self.sess = None
        self.depth_dict = depth_dict
        self.url_list = ["http://en.wikipedia.org/wiki/List_of_maritime_disasters",
                    "http://en.wikipedia.org/wiki/2013_Lampedusa_migrant_shipwreck",
                    "http://www.euronews.com/2018/07/19/-open-arms-migrant-rescue-boat-heads-for-spain-amid-row-with-italy",
                    "https://www.thelocal.it/20180630/spain-ngo-ship-saves-59-migrants-italy-refuses-access",
                    "https://www.opendemocracy.net/beyondslavery/martina-tazzioli/open-cities-open-harbours",
                    "http://www.thetablet.co.uk/news/9447/pope-francis-in-new-plea-for-migrant-safety-",
                    "https://www.vanguardngr.com/2018/07/italian-tugboat-with-66-rescued-migrants-refused-entry-into-italian-port/",
                    "https://www.newsdeeply.com/refugees/community/2018/07/02/e-u-deportations-risk-further-destabilizing-mali",
                   ]
        if self.depth_dict == {}:
            self.depth_dict = {"http://en.wikipedia.org/wiki/List_of_maritime_disasters":0,
                          "http://en.wikipedia.org/wiki/2013_Lampedusa_migrant_shipwreck":0, 
                          "http://www.euronews.com/2018/07/19/-open-arms-migrant-rescue-boat-heads-for-spain-amid-row-with-italy":0, 
                          "https://www.thelocal.it/20180630/spain-ngo-ship-saves-59-migrants-italy-refuses-access":0,
                          "https://www.opendemocracy.net/beyondslavery/martina-tazzioli/open-cities-open-harbours":0,
                          "http://www.thetablet.co.uk/news/9447/pope-francis-in-new-plea-for-migrant-safety-":0,
                          "https://www.vanguardngr.com/2018/07/italian-tugboat-with-66-rescued-migrants-refused-entry-into-italian-port/":0,
                          "https://www.newsdeeply.com/refugees/community/2018/07/02/e-u-deportations-risk-further-destabilizing-mali":0             
                         }
    
    ## Add seed URLs to Ordered Dictionary
    def create_dict(self):
        for url in self.url_list:
            self.queue_dict[url] = True
        return
    
    ## Save the data in XML format
    def save_xml(self, docno, http_header, title, text, html_source, author, depth, url):  
        item_tree = ET.Element("items")
        doc = ET.SubElement(item_tree,"item")
        docno_tree = ET.SubElement(doc, "docno")
        http_header_tree = ET.SubElement(doc, "http_header")
        title_tree = ET.SubElement(doc, "title")
        text_tree = ET.SubElement(doc, "text")
        html_source_tree = ET.SubElement(doc, "html_source")
        author_tree = ET.SubElement(doc, "author")
        depth_tree = ET.SubElement(doc, "depth")
        url_tree = ET.SubElement(doc, "url")
        
        try:
            ET.SubElement(docno_tree, "value").text = str(docno)
            ET.SubElement(http_header_tree,"value").text = str(http_header)
            ET.SubElement(title_tree, "value").text = str(title)
            ET.SubElement(text_tree, "value").text = str(text)
            ET.SubElement(html_source_tree, "value").text = str(html_source)
            ET.SubElement(author_tree, "value").text = str(author)
            ET.SubElement(depth_tree,"value").text = str(depth)
            ET.SubElement(url_tree,"value").text = str(url)
        except:
            docno = re.sub(u'[^\u0020-\uD7FF\u0009\u000A\u000D\uE000-\uFFFD\U00010000-\U0010FFFF]+', '', str(docno))
            http_header = re.sub(u'[^\u0020-\uD7FF\u0009\u000A\u000D\uE000-\uFFFD\U00010000-\U0010FFFF]+', '', str(http_header))
            title = re.sub(u'[^\u0020-\uD7FF\u0009\u000A\u000D\uE000-\uFFFD\U00010000-\U0010FFFF]+', '', str(title))
            text = re.sub(u'[^\u0020-\uD7FF\u0009\u000A\u000D\uE000-\uFFFD\U00010000-\U0010FFFF]+', '', str(text))
            html_source = re.sub(u'[^\u0020-\uD7FF\u0009\u000A\u000D\uE000-\uFFFD\U00010000-\U0010FFFF]+', '', str(html_source))
            author = re.sub(u'[^\u0020-\uD7FF\u0009\u000A\u000D\uE000-\uFFFD\U00010000-\U0010FFFF]+', '', str(author))
            depth = re.sub(u'[^\u0020-\uD7FF\u0009\u000A\u000D\uE000-\uFFFD\U00010000-\U0010FFFF]+', '', str(depth))
            url = re.sub(u'[^\u0020-\uD7FF\u0009\u000A\u000D\uE000-\uFFFD\U00010000-\U0010FFFF]+', '', str(url))
            
            ET.SubElement(docno_tree, "value").text = str(docno)
            ET.SubElement(http_header_tree,"value").text = str(http_header)
            ET.SubElement(title_tree, "value").text = str(title)
            ET.SubElement(text_tree, "value").text = str(text)
            ET.SubElement(html_source_tree, "value").text = str(html_source)
            ET.SubElement(author_tree, "value").text = str(author)
            ET.SubElement(depth_tree,"value").text = str(depth)
            ET.SubElement(url_tree,"value").text = str(url)

        tree = ET.ElementTree(item_tree)
        filename = "./data_files/" + str(self.counter)+".xml"
        tree.write(filename)
        self.counter += 1
        return
    
    ## Function to get the valid text out of an html page
    def tag_visible(self, element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True
    
    ## Update outlink
    def update_outlink(self, current_url, outlink_url):
        if current_url not in self.url_out_link_dict:
            self.url_out_link_dict[current_url] = [outlink_url]
        else:
            self.url_out_link_dict[current_url].append(outlink_url) 
            
    ## Update outlink
    def update_inlink(self, current_url, inlink_url):
        if inlink_url not in self.url_in_link_dict:
            self.url_in_link_dict[inlink_url] = [current_url]
        else:
            self.url_in_link_dict[inlink_url].append(current_url) 
            
    ## Politeness policy check and introduce time delay between same host name
    def check_time_delay(self, current_url, previous_url, previous_time = None):
        if current_url == previous_url:
            if previous_time == None:
                time_to_sleep = 0.3
            else:
                current_time = time.time()
                diff = current_time - previous_time
                if diff < 0.5:
                    time_to_sleep = (0.5 - diff)
                else:
                    return True
            time.sleep(time_to_sleep) ## give 1 sec delay
        return True

    ## Manage the frontier by promoting the URLs with highest inlinks after every 100 crawls
    def frontier_manager(self):
        self.max_inlink = list(self.url_in_link_dict.keys())[0]
        for link_in_queue in self.queue_dict:
            if link_in_queue in self.url_in_link_dict:
                if len(self.url_in_link_dict[self.max_inlink]) <= len(self.url_in_link_dict[link_in_queue]):
                    self.max_inlink = link_in_queue
        if self.max_inlink not in self.queue_dict:
            return 
        return self.max_inlink
    
    ## Call made to frontier manager every 100th iteration and popping from ordered dict
    def pop_from_queue(self):
        if self.counter % 100 == 0:
            self.dump_files()
            url = self.frontier_manager()
            if url:
                print ("url_to_be_removed",url)
                self.queue_dict.pop(url)
            else:
                url = self.queue_dict.popitem(last = False)[0]
        else:
            url = self.queue_dict.popitem(last = False)[0]
        self.last_url_seen = self.page_parser(url)
        self.queue_dict_crawled[url] = True
        print ("Number:",self.counter ,"Crawled:",url, "Total in queue", (len(self.queue_dict)))
        return self.last_url_seen
    
    ## Start the crawler
    def start_crawler (self):
        if len(self.queue_dict) == 0:
            self.create_dict()
        self.sess = requests.Session()
        a = requests.adapters.HTTPAdapter(max_retries=3)
        b = requests.adapters.HTTPAdapter(max_retries=3)
        self.sess.mount('http://', a)
        self.sess.mount('https://', b)
        while (len(self.queue_dict) == 0 or self.counter < 25001):
            self.last_url_seen = self.pop_from_queue()
            
    ## Get only the host and domain name
    def get_domain_name(self, url):
        urlinfo = urlparse(url)
        url_main = urlinfo[0] + "://" + urlinfo[1]
        return url_main
    
    ## Main function to parse a page
    def get_parse(self, url, parse = True):
        try:
            page = self.sess.get(url, timeout = 3)
            page_header = page.headers
            if parse:
                page_parser = BeautifulSoup(page.text, "html.parser")
                page_title = page_parser.title.text
                texts = page_parser.findAll(text=True)
                visible_texts = filter(self.tag_visible, texts)  
                page_content_text = u" ".join(t.strip() for t in visible_texts)
                return page_parser, page_header, page_title, page_content_text
            else:
                return page_header
        except Exception as e:
            print ("Exception", e, "url", url)
            return None, None, None, None
        
    ## Check the robot.txt file 
    def can_fetch_url(self, url):
        try:
            if self.dict_domain_check[url]:
                pass
        except:
            try:
                self.dict_domain_check[url] = False
                access = self.sess.get(url, timeout = 2)
                self.rp = urllib.robotparser.RobotFileParser()
                robots_txt = url + "/robots.txt"
                self.rp.set_url(robots_txt)
                self.rp.read()
                status = self.rp.can_fetch("*", url)   
                self.dict_domain_check[url] = status
                return self.dict_domain_check[url]
            except Exception as e:
                print ("Following except encountered", e, "url", url)
                self.dict_domain_check[url] = False
                return self.dict_domain_check[url]
        return self.dict_domain_check[url]
    
    ## Parse the page using beautiful soup and get all needed content for saving in Elastic Search
    def page_parser(self, url):
        t1 = time.time()
        current_domain_name = self.get_domain_name(url)
        self.check_time_delay(current_domain_name, self.last_seen_url)
        page_parser, page_header, page_title, page_content_text  = self.get_parse(url)
        if not page_parser:
            return 
        page_reader = page_parser.find_all(name = "a", href = True)
        previous_link = current_domain_name
        canonicalized_url = None
        current_url_dict = {}
        current_domain_name = self.check_validity_of_url(page_reader, url, previous_link)
        if page_content_text:
            self.save_xml(docno = url, http_header = page_header, title = page_title, text = page_content_text, html_source = page_parser.html, author = "Monica", depth = self.depth_dict[url], url = url)
        t2 = time.time()
        print ("Time taken:", t2-t1)
    
    ## Check for all conditions and then add URLs to queue 
    def check_validity_of_url(self, page_reader, url, previous_link):
        current_domain_name = previous_link
        t1 = time.time()
        print ("Currently crawling for: ", url)
        for link in page_reader:
            if self.counter < 25001:
                current_url = link["href"]
                if "http" in current_url or "wiki" in current_url:
                    if "wiki" in current_url and ".org" not in current_url:
                        current_url = "https://en.wikipedia.org" + current_url
                    canonicalized_url = w3lib.url.canonicalize_url(current_url)
                    self.update_inlink(url, canonicalized_url)
                    self.update_outlink(url, canonicalized_url)
                    if invalid_extension().valid(canonicalized_url) and url != canonicalized_url:
                        try:
                            if self.queue_dict[canonicalized_url]:
                                continue
                        except KeyError:
                            try:
                                if self.queue_dict_crawled[canonicalized_url]:
                                    continue
                            except KeyError:
                                anchor_text = canonicalized_url + link.text
                                if invalid_extension().keyword_presence(anchor_text):
                                    current_domain_name = self.get_domain_name(canonicalized_url)
                                    if self.can_fetch_url(current_domain_name):
                                        self.check_time_delay(current_domain_name, previous_link, t1)
                                        t1 = time.time()
                                        current_page_header = self.get_parse(canonicalized_url, parse = False)
                                        previous_link = current_domain_name
                                        try:
                                            content_type = current_page_header["Content-Type"]
                                        except:
                                            continue
                                        try:
                                            content_language = current_page_header["Content-Language"]
                                        except:
                                            content_language = "en"                                   
                                        if "text/html" in content_type and "en" in content_language:
                                            self.queue_dict[canonicalized_url] = True
                                            self.depth_dict[canonicalized_url] = self.depth_dict[url] + 1
                                            print ("URL Appended:",canonicalized_url)
        return current_domain_name
    
    ## Temporary save the needed data structures in system using pickle
    def dump_files(self):
        with open("queue_dict","wb") as file:
            pickle.dump(self.queue_dict, file)

        with open("queue_dict_crawled","wb") as file:
            pickle.dump(self.queue_dict_crawled, file)

        with open("last_seen_url","wb") as file:
            pickle.dump(self.last_seen_url, file)
            
        with open("dic","wb") as file:
            pickle.dump(self.last_seen_url, file)

        with open("counter","wb") as file:
            pickle.dump(self.counter, file)

        with open("url_out_link_dict","wb") as file:
            pickle.dump(self.url_out_link_dict, file)

        with open("url_in_link_dict","wb") as file:
            pickle.dump(self.url_in_link_dict, file)

        with open("max_inlink","wb") as file:
            pickle.dump(self.max_inlink, file)

        with open("dict_domain_check","wb") as file:
            pickle.dump(self.dict_domain_check, file)
            
        with open("depth_dict","wb") as file:
            pickle.dump(self.depth_dict, file)  

        

In [43]:
crawler_class_object = crawler_class(counter, url_out_link_dict, url_in_link_dict, depth_dict, queue_dict, queue_dict_crawled ,last_seen_url, max_inlink, dict_domain_check)


In [45]:
crawler_class_object.start_crawler()

url_to_be_removed https://shipoftheday.blogspot.com/search/label/Eindhoven
Currently crawling for:  https://shipoftheday.blogspot.com/search/label/Eindhoven
Time taken: 2.532471179962158
Number: 21001 Crawled: https://shipoftheday.blogspot.com/search/label/Eindhoven Total in queue 53032
Currently crawling for:  https://en.wikipedia.org/wiki/French_ship_Alg%C3%A9siras_(1855)
Time taken: 0.30817747116088867
Number: 21002 Crawled: https://en.wikipedia.org/wiki/French_ship_Alg%C3%A9siras_(1855) Total in queue 52817
Currently crawling for:  http://www.tynebuiltships.co.uk/index.html
Time taken: 0.3552727699279785
Number: 21003 Crawled: http://www.tynebuiltships.co.uk/index.html Total in queue 52816
Currently crawling for:  http://www.tynebuiltships.co.uk/Shipbuilders.html
Time taken: 0.1316847801208496
Number: 21004 Crawled: http://www.tynebuiltships.co.uk/Shipbuilders.html Total in queue 52815
Currently crawling for:  http://www.tynebuiltships.co.uk/Ships.html
Time taken: 0.117685317993164

Currently crawling for:  https://en.wikipedia.org/wiki/Moiety_(kinship)
Time taken: 0.36676764488220215
Number: 21041 Crawled: https://en.wikipedia.org/wiki/Moiety_(kinship) Total in queue 52783
Currently crawling for:  https://en.wikipedia.org/wiki/Joking_relationship
Time taken: 0.3957479000091553
Number: 21042 Crawled: https://en.wikipedia.org/wiki/Joking_relationship Total in queue 52782
Currently crawling for:  http://eclectic.ss.uci.edu/~drwhite/tlaxcala/
Time taken: 0.8454358577728271
Number: 21043 Crawled: http://eclectic.ss.uci.edu/~drwhite/tlaxcala/ Total in queue 52781
Currently crawling for:  https://en.wikipedia.org/wiki/Fictive_kinship
Time taken: 0.7273883819580078
Number: 21044 Crawled: https://en.wikipedia.org/wiki/Fictive_kinship Total in queue 52780
Currently crawling for:  https://en.wikipedia.org/wiki/Milk_kinship
Time taken: 0.11970758438110352
Number: 21045 Crawled: https://en.wikipedia.org/wiki/Milk_kinship Total in queue 52779
Currently crawling for:  https://e

Time taken: 0.20594024658203125
Number: 21070 Crawled: http://www.umanitoba.ca/faculties/arts/anthropology/tutor/case_studies/turkish/turkterm.html Total in queue 52761
Currently crawling for:  https://en.wikipedia.org/wiki/Sudanese_kinship
Time taken: 0.09726572036743164
Number: 21071 Crawled: https://en.wikipedia.org/wiki/Sudanese_kinship Total in queue 52760
Currently crawling for:  https://en.wikipedia.org/wiki/Warlpiri_people
Time taken: 0.2155933380126953
Number: 21072 Crawled: https://en.wikipedia.org/wiki/Warlpiri_people Total in queue 52759
Currently crawling for:  https://en.wikipedia.org/wiki/Australian_Aboriginal_kinship
Time taken: 0.2692561149597168
Number: 21073 Crawled: https://en.wikipedia.org/wiki/Australian_Aboriginal_kinship Total in queue 52758
Currently crawling for:  https://en.wikipedia.org/wiki/Burmese_kinship
Time taken: 0.17442631721496582
Number: 21074 Crawled: https://en.wikipedia.org/wiki/Burmese_kinship Total in queue 52757
Currently crawling for:  http:/

Currently crawling for:  https://en.wikiversity.org/wiki/interpersonal_relationships
URL Appended: http://7125-6666.blogspot.com/2007/09/relationship-video-1-vodafone-aeroplane.html
URL Appended: http://7125-6666.blogspot.com/2007/09/relationship-video-2-last-rolo.html
URL Appended: http://7125-6666.blogspot.com/2007/09/relationship-video-3-blind-date.html
URL Appended: http://7125-6666.blogspot.com/2007/09/relationship-video-4-toy-boy-hyundai.html
URL Appended: https://en.wikipedia.org/wiki/Happiness_and_Life
URL Appended: https://en.wikipedia.org/wiki/Social_psychology_(psychology)/Lectures/Relationships
URL Appended: https://en.wikipedia.org/wiki/User:Jenny_O/Relationships
Time taken: 3.5957818031311035
Number: 21092 Crawled: https://en.wikiversity.org/wiki/interpersonal_relationships Total in queue 52781
Currently crawling for:  https://en.wikipedia.org/wiki/Domestic_partnership
URL Appended: https://en.wikipedia.org/wiki/Right_of_survivorship
URL Appended: https://en.wikipedia.org

Currently crawling for:  https://en.wikipedia.org/wiki/Employment
URL Appended: https://en.wikipedia.org/wiki/Motor_vehicle_accidents
URL Appended: https://en.wikipedia.org/wiki/Workplace_friendship
URL Appended: https://en.wikipedia.org/wiki/Workplace_relationships
Time taken: 1.9658777713775635
Number: 21105 Crawled: https://en.wikipedia.org/wiki/Employment Total in queue 52830
Currently crawling for:  https://en.wikipedia.org/wiki/Relationship_breakup
Time taken: 0.14652347564697266
Number: 21106 Crawled: https://en.wikipedia.org/wiki/Relationship_breakup Total in queue 52829
Currently crawling for:  https://en.wikipedia.org/wiki/Relationship_counseling
URL Appended: https://en.wikipedia.org/wiki/Sexual_relationship_disorder
Time taken: 0.8193590641021729
Number: 21107 Crawled: https://en.wikipedia.org/wiki/Relationship_counseling Total in queue 52829
Currently crawling for:  https://en.wikipedia.org/wiki/Internet_relationship
Time taken: 0.2374260425567627
Number: 21108 Crawled: ht

...

Currently crawling for:  https://en.wikipedia.org/wiki/P-Funk_Mothership
URL Appended: https://nmaahc.si.edu/explore/collection/search?edan_local=1&edan_q=Mothership&op=Search
URL Appended: http://www.smithsonianmag.com/smithsonian-institution/watch-george-clintons-mothership-get-reassembled-for-its-museum-debut-180951685/
URL Appended: https://www.washingtonpost.com/wp-dyn/content/article/2010/04/11/AR2010041103996.html
URL Appended: http://blog.washingtonpost.com/clicktrack/2010/04/b-sides_george_clinton_bernie.html?sid=ST2010041200169
Time taken: 5.43297815322876
Number: 24919 Crawled: https://en.wikipedia.org/wiki/P-Funk_Mothership Total in queue 62525
Currently crawling for:  https://en.wikipedia.org/wiki/Mothership_(composition)
URL Appended: http://www.masonbates.com/work/work-mothership.html
Time taken: 2.3923516273498535
Number: 24920 Crawled: https://en.wikipedia.org/wiki/Mothership_(composition) Total in queue 62525
Currently crawling for:  https://en.wikipedia.org/wiki/Mo_T

Currently crawling for:  https://en.wikipedia.org/wiki/SpaceShipThree
URL Appended: http://www.flightglobal.com/articles/2005/08/23/201097/spaceshipthree-poised-to-follow-if-ss2-succeeds.html
URL Appended: http://www.space.com/2728-burt-rutan-civilian-spaceflight-breakthroughs-spaceshiptwo.html
URL Appended: http://www.flightglobal.com/blogs/hyperbola/2008/02/spaceshipthree-revealed.html
Time taken: 4.606718301773071
Number: 24947 Crawled: https://en.wikipedia.org/wiki/SpaceShipThree Total in queue 62517
Currently crawling for:  http://www.astronautix.com/s/spaceshipone.html
URL Appended: http://space.skyrocket.de/doc_sat/ss1_hist.htm
Time taken: 0.8589389324188232
Number: 24948 Crawled: http://www.astronautix.com/s/spaceshipone.html Total in queue 62517
Currently crawling for:  http://www.scaled.com/projects/tierone/combined_white_knight_spaceshipone_flight_tests
URL Appended: https://www.scaled.com/portfolio/starship/
URL Appended: https://www.scaled.com/portfolio/spaceshipone/
Time 

URL Appended: https://gizmodo.com/virgin-galactic-announces-new-cosmic-girl-mothership-1746068379
URL Appended: http://news.bbc.co.uk/2/hi/science/nature/3712998.stm
URL Appended: http://www.soultek.com/space_privatization/Virgin_Galactic_Captain_Kirk_and_spaceshipone.htm
URL Appended: https://online.wsj.com/articles/problems-plagued-virgin-galactic-rocket-ship-long-before-crash-1415838171
URL Appended: http://spacefellowship.com/News/?p=7772
URL Appended: http://www.space.com/1439-okays-virgin-galactic-spaceship-plans.html
URL Appended: http://spacefellowship.com/News/?p=6155
Time taken: 16.657195329666138
Number: 24963 Crawled: https://en.wikipedia.org/wiki/Virgin_SpaceShip Total in queue 62561
Currently crawling for:  https://en.wikipedia.org/wiki/The_Spaceship_Company_Tier_1b
URL Appended: https://en.wikipedia.org/wiki/Scaled_Composites_SpaceShipTwo
URL Appended: https://www.forbes.com/feeds/ap/2007/07/20/ap3937696.html
Time taken: 1.7402386665344238
Number: 24964 Crawled: https://

Currently crawling for:  https://www.bbc.co.uk/news/science-environment-20500194
URL Appended: http://www.bbc.co.uk/news/mobile/science-environment-11511604
Time taken: 1.7427351474761963
Number: 24978 Crawled: https://www.bbc.co.uk/news/science-environment-20500194 Total in queue 62587
Currently crawling for:  https://www.nytimes.com/2014/11/01/science/virgin-galactics-spaceshiptwo-crashes-during-test-flight.html
URL Appended: http://www.nytimes.com/2013/04/30/science/space/virgin-galactics-spaceshiptwo-inches-closer-to-space.html
URL Appended: https://www.nytimes.com/2013/04/30/science/space/virgin-galactics-spaceshiptwo-inches-closer-to-space.html?action=click&module=RelatedCoverage&pgtype=Article&region=Footer
Time taken: 1.8322348594665527
Number: 24979 Crawled: https://www.nytimes.com/2014/11/01/science/virgin-galactics-spaceshiptwo-crashes-during-test-flight.html Total in queue 62588
Currently crawling for:  http://www.spacenews.com/article/launch-report/42391spaceshiptwo-destro

URL Appended: https://www.space.com/27376-virgin-galactic-spaceshiptwo-test-flight.html
Time taken: 1.4169883728027344
Number: 24998 Crawled: http://www.space.com/27376-virgin-galactic-spaceshiptwo-test-flight.html Total in queue 62593
Currently crawling for:  http://cosmiclog.nbcnews.com/_news/2012/12/19/16025374-spaceshiptwo-straps-on-its-engine
URL Appended: http://www.virgingalactic.com/news/item/spaceshiptwo-completes-first-glide-in-powered-flight-configuration/
URL Appended: http://cosmiclog.nbcnews.com/_news/2012/10/22/14623551-jeff-bezos-blue-origin-spaceship-company-aces-pad-escape-test?lite=
Time taken: 1.3233647346496582
Number: 24999 Crawled: http://cosmiclog.nbcnews.com/_news/2012/12/19/16025374-spaceshiptwo-straps-on-its-engine Total in queue 62594
Currently crawling for:  http://www.virgingalactic.com/news/item/virgin-galactic-breaks-speed-of-sound-in-first-rocket-powered-flight-of-spaceshiptwo/
Time taken: 0.28725409507751465
Number: 25000 Crawled: http://www.virgingala

In [46]:
## Load the file from disk using pickle
def load_file():
    with open("queue_dict","rb") as file:
        queue_dict = pickle.load(file)

    with open("queue_dict_crawled","rb") as file:
        queue_dict_crawled = pickle.load(file)

    with open("last_seen_url","rb") as file:
        last_seen_url = pickle.load(file)

    with open("counter","rb") as file:
        counter = pickle.load(file)

    with open("url_out_link_dict","rb") as file:
        url_out_link_dict = pickle.load(file)

    with open("url_in_link_dict","rb") as file:
        url_in_link_dict = pickle.load(file)

    with open("max_inlink","rb") as file:
        max_inlink = pickle.load(file)

    with open("dict_domain_check","rb") as file:
        dict_domain_check = pickle.load(file) 
        
    with open("depth_dict","rb") as file:
        depth_dict = pickle.load(file)
    return queue_dict, queue_dict_crawled, last_seen_url, counter, url_out_link_dict, url_in_link_dict, max_inlink, dict_domain_check, depth_dict

In [47]:
## call load function
queue_dict, queue_dict_crawled, last_seen_url, counter, url_out_link_dict, url_in_link_dict, max_inlink, dict_domain_check, depth_dict = load_file()